In [14]:
import mowl

mowl.init_jvm("10g")

from de.tudresden.inf.lat.jcel.ontology.normalization import OntologyNormalizer, SimpleNormalizer
from de.tudresden.inf.lat.jcel.ontology.axiom.extension import IntegerOntologyObjectFactoryImpl
from de.tudresden.inf.lat.jcel.owlapi.translator import ReverseAxiomTranslator, Translator
from de.tudresden.inf.lat.jcel.ontology.axiom.complex import IntegerInverseObjectPropertiesAxiom
from de.tudresden.inf.lat.jcel.coreontology.axiom import NormalizedIntegerAxiom
from org.semanticweb.owlapi.model.parameters import Imports
from java.util import HashSet
import logging
logging.basicConfig(level=logging.INFO)
from mowl.owlapi import OWLAPIAdapter
from mowl.datasets import PathDataset
from mowl.ontology.normalize import process_axiom

Let's have a look at one small ontology consisting of just one complex axiom:

In [ ]:
small_ontology = PathDataset("small_ontology.owl").ontology

Here's the list of axioms:

In [4]:
for ax in list(small_ontology.getAxioms()):
    print(str(ax))

Declaration(Class(<B>))
Declaration(Class(<A>))
Declaration(Class(<D>))
Declaration(ObjectProperty(<s>))
Declaration(Class(<C>))
Declaration(ObjectProperty(<r>))
EquivalentClasses(<A> ObjectIntersectionOf(<B> ObjectSomeValuesFrom(<r> ObjectIntersectionOf(<C> ObjectSomeValuesFrom(<s> <D>)))))


The code below follows a general procedure for ontology normalization (see [here](https://github.com/bio-ontology-research-group/mowl/blob/main/mowl/ontology/normalize.py)) <br> <br>
``Translator`` is used for converting object properties and concept names to integer numbers; normalization is applied to a modified ontology

In [5]:
translator = Translator(small_ontology.getOWLOntologyManager().getOWLDataFactory(), IntegerOntologyObjectFactoryImpl())
small_axioms = HashSet()
small_axioms.addAll(small_ontology.getAxioms())
translator.getTranslationRepository().addAxiomEntities(small_ontology)

for o in small_ontology.getImportsClosure():
    small_axioms.addAll(o.getAxioms())
    translator.getTranslationRepository().addAxiomEntities(o)

intAxioms = translator.translateSA(small_axioms)

Note that ``Translator`` encodes concept names and role names only from the ontology and doesn't take into consideration new concept names. Concept names (different from ``owl:Thing`` and ``owl:Nothing``) are encoded starting from number ``6`` up to ``# of concept names + 4`` (including ``owl:Thing`` and ``owl:Nothing``) according to class declarations; this number is chosen because integers from ``0`` to ``5`` are reserved for bottom and top concepts, bottom and top object and data properties (see [here](https://github.com/julianmendez/jcel/blob/f37f7f6701b58c266aa95528512e73d9f5516644/jcel-coreontology/src/main/java/de/tudresden/inf/lat/jcel/coreontology/datatype/IntegerEntityManager.java#L60)). Role names are encoded starting from ``# of concept names + 5`` up to ``# of concept names + 4 + # of role names``. <br> <br>
``intAxioms`` contains encoded ontology axioms:

In [6]:
for ax in list(intAxioms):
    print(str(ax))

Declaration(Class(6))
Declaration(Class(7))
Declaration(Class(8))
Declaration(Class(9))
Declaration(ObjectProperty(10))
Declaration(ObjectProperty(11))
EquivalentClasses(6 ObjectIntersectionOf(ObjectSomeValuesFrom(10 ObjectIntersectionOf(ObjectSomeValuesFrom(11 9) 8)) 7) )


``IntegerOntologyObjectFactoryImpl`` is a factory for generating 'new' concept and role names appearing during normalization procedure rith the help of ``IntegerEntityManager``. Note that the first identifier used is set to integer ``6`` (see [here](https://github.com/julianmendez/jcel/blob/f37f7f6701b58c266aa95528512e73d9f5516644/jcel-coreontology/src/main/java/de/tudresden/inf/lat/jcel/coreontology/datatype/IntegerEntityManager.java#L63) and [here](https://github.com/julianmendez/jcel/blob/f37f7f6701b58c266aa95528512e73d9f5516644/jcel-coreontology/src/main/java/de/tudresden/inf/lat/jcel/coreontology/datatype/IntegerEntityManagerImpl.java#L76)), and then iterator increases the counter by ``1`` (see [here](https://github.com/julianmendez/jcel/blob/f37f7f6701b58c266aa95528512e73d9f5516644/jcel-coreontology/src/main/java/de/tudresden/inf/lat/jcel/coreontology/datatype/IntegerEntityManagerImpl.java#L98)), i.e., it iterates first over concept names from the signature.

In [9]:
factory = IntegerOntologyObjectFactoryImpl()

# axiom normalizer
n = SimpleNormalizer(factory)

# object properties from the signature encoded
objectPropIdSet = HashSet()
# in this axiom set axioms need processing are stored
currentAxiomSet = HashSet()

# first process inverse object properties axioms; we don't have them in our ontology
for ax in intAxioms:
    objectPropIdSet.addAll(ax.getObjectPropertiesInSignature())
    if isinstance(ax, IntegerInverseObjectPropertiesAxiom):
        newSet = n.normalize(axiom)
        currentAxiomSet.addAll(newSet)
    else:
        currentAxiomSet.add(ax)

Here inverse object property ids are generated; note that they start from ``6``, the first usable identifier. Since we don't have inverse property axioms, no normalized axioms are outputed here. 

In [10]:
for propId in list(objectPropIdSet): 
	print(f'object property id = {propId}')
	inversePropId = factory.getEntityManager().createOrGetInverseObjectPropertyOf(propId)
	print(f'corresponding inverse object property id = {inversePropId}')
	print(f'the corresponding axiom = {n.getAxiomsForInverseObjectProperties(propId, inversePropId)}')
	currentAxiomSet.addAll(n.getAxiomsForInverseObjectProperties(propId, inversePropId))

object property id = 10
corresponding inverse object property id = 6
the corresponding axiom = []
object property id = 11
corresponding inverse object property id = 7
the corresponding axiom = []


Let's have a look at the whole normalization procedure. Declaration axioms are ignored. Note that newly sampled concept identifiers start from ``8`` since ``6`` and ``7`` were reserved for inverse object properties during previous steps of normalization algorithm: <br> <br>

``not normalized axiom: SubClassOf(ObjectIntersectionOf(ObjectSomeValuesFrom(10 ObjectIntersectionOf(ObjectSomeValuesFrom(11 9) 8)) 7) 6)`` <br>
``The axiom is replaced with the following set of axioms:`` <br>
``['SubClassOf(ObjectSomeValuesFrom(10 ObjectIntersectionOf(ObjectSomeValuesFrom(11 9) 8)) 8)', 'SubClassOf(ObjectIntersectionOf(7 8) 6)']`` <br> <br>

Also note that concept name identifier counter outputs ids which exceed the range of concept identifiers used in the signature (we have 4 concept names, they have identifiers ``6``, ``7``, ``8``, ``9``). This will later make it impossible to convert them into valid OWL SubClassOf axioms. 

In [12]:
# the set of normalized axioms
ret = HashSet()
# until the list of axioms need processing is empty
while currentAxiomSet.size() > 0:
    print("===================")
    nextAxiomSet = HashSet()
    for axiom in list(currentAxiomSet):
        if isinstance(axiom, NormalizedIntegerAxiom):
            print(f'normalized axiom: {axiom}')
            ret.add(axiom)
        else:
            print(f'not normalized axiom: {axiom}')
            newSet = n.normalize(axiom)
            print('The axiom is replaced with the following set of axioms:')
            print([str(elem) for elem in list(newSet)])
            nextAxiomSet.addAll(newSet)
    currentAxiomSet = nextAxiomSet

not normalized axiom: Declaration(Class(6))
The axiom is replaced with the following set of axioms:
[]
not normalized axiom: Declaration(Class(7))
The axiom is replaced with the following set of axioms:
[]
not normalized axiom: Declaration(Class(8))
The axiom is replaced with the following set of axioms:
[]
not normalized axiom: Declaration(Class(9))
The axiom is replaced with the following set of axioms:
[]
not normalized axiom: Declaration(ObjectProperty(10))
The axiom is replaced with the following set of axioms:
[]
not normalized axiom: Declaration(ObjectProperty(11))
The axiom is replaced with the following set of axioms:
[]
not normalized axiom: EquivalentClasses(6 ObjectIntersectionOf(ObjectSomeValuesFrom(10 ObjectIntersectionOf(ObjectSomeValuesFrom(11 9) 8)) 7) )
The axiom is replaced with the following set of axioms:
['SubClassOf(ObjectIntersectionOf(ObjectSomeValuesFrom(10 ObjectIntersectionOf(ObjectSomeValuesFrom(11 9) 8)) 7) 6)', 'SubClassOf(6 ObjectIntersectionOf(ObjectSom

Here's the output of normalization algorithm:

In [13]:
for ax in list(ret):
    print(str(ax))

SubClassOf*(9 8)
SubClassOf*(ObjectIntersectionOf(8 11) 10)
SubClassOf*(6 ObjectSomeValuesFrom(10 9))
SubClassOf*(9 ObjectSomeValuesFrom(11 9))
SubClassOf*(ObjectSomeValuesFrom(10 10) 8)
SubClassOf*(ObjectIntersectionOf(7 8) 6)
SubClassOf*(ObjectSomeValuesFrom(11 9) 11)
SubClassOf*(6 7)


Now we need to apply ``ReverseAxiomTranslator`` to get from integer-valued axioms to 'normal' ones. Here you can see that translation map breaks on axioms where new concept names are used (identifiers ``10`` and ``11``). So, the resulting normalized ontology will be missing them. 

In [15]:
rTranslator = ReverseAxiomTranslator(translator, small_ontology)
for ax in list(ret):
    try:
        axiom = rTranslator.visit(ax)
        key, value = process_axiom(axiom)
    except Exception as e:
        print(f'not processed axiom: {str(ax)}')
        logging.info("Reverse translation. Ignoring axiom: %s", ax)
        logging.info(e)

INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectIntersectionOf(8 11) 10)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '11'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectSomeValuesFrom(10 10) 8)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '10'.
INFO:root:Reverse translation. Ignoring axiom: SubClassOf*(ObjectSomeValuesFrom(11 9) 11)
INFO:root:de.tudresden.inf.lat.jcel.owlapi.translator.TranslationException: The translation map is incomplete. Item id was not found: '11'.


not processed axiom: SubClassOf*(ObjectIntersectionOf(8 11) 10)
not processed axiom: SubClassOf*(ObjectSomeValuesFrom(10 10) 8)
not processed axiom: SubClassOf*(ObjectSomeValuesFrom(11 9) 11)
